# Kapitel 7 – Klassen und soziale Ordnung in Rom

In diesem Notebook modellieren wir mit Python Klassen die römische Sozialstruktur:

- freie Bürgerinnen und Bürger
- Sklavinnen und Sklaven
- Freigelassene (*liberti* / *libertini*)

und zeigen wie sich der Name eines Sklaven bei der Freilassung korrekt verändert.


## 1. Warum war die alte Darstellung der *libertini* falsch?

In vielen vereinfachten Modellen wird so getan als seien **libertini** eine völlig eigene
soziale Kategorie neben freien Römern und Sklaven. Historisch ist das nur halb richtig:

- Ein **libertus** ist konkret der Freigelassene eines bestimmten Patrons
  (z. B. *Marci libertus* = Freigelassener des Marcus).
- Ein **libertinus** ist allgemein ein früherer Sklave; seine Kinder sind bereits
  **ingenui**, also voll freie Bürger.
- Freigelassene bleiben rechtlich und patronal mit ihrem ehemaligen Herrn verbunden
  (Klientelbindung, Pflichten), sind aber **keine Sklaven mehr**.

Ein sauberes Modell braucht daher:

1. Eine Basisklasse für Menschen
2. Eine Klasse für freie Bürger
3. Eine Klasse für Sklaven mit Verweis auf den Herrn
4. Eine Klasse für Freigelassene mit korrekter Namensformel und Patron

Genau das setzen wir jetzt um.

In [ ]:
# Hilfsfunktion: Praenomen in den Genitiv setzen (stark vereinfacht)

PRAENOMEN_GENITIVE = {
    "Marcus": "Marci",
    "Gaius": "Gai",
    "Lucius": "Luci",
    "Publius": "Publi",
    "Quintus": "Quinti",
    "Titus": "Titi",
    "Sextus": "Sexti",
    "Aulus": "Auli"
}

def praenomen_genitive(praenomen: str) -> str:
    """Gibt den (vereinfachten) Genitiv eines Praenomens zurück."""
    return PRAENOMEN_GENITIVE.get(praenomen, praenomen + "i")

In [ ]:
class Homo:
    """Basisklasse für Menschen im römischen Kontext."""

    def __init__(self, praenomen=None, nomen=None, cognomen=None):
        self.praenomen = praenomen
        self.nomen = nomen
        self.cognomen = cognomen

    @property
    def status(self) -> str:
        return "homo"

    @property
    def full_name(self) -> str:
        parts = [p for p in [self.praenomen, self.nomen, self.cognomen] if p]
        return " ".join(parts) if parts else "(namenlos)"

    def __repr__(self):
        return f"<{self.__class__.__name__}: {self.full_name} ({self.status})>"

In [ ]:
class Citizen(Homo):
    """Freier römischer Bürger (ingenui oder später auch libertini)."""

    @property
    def status(self) -> str:
        return "civis Romanus"

In [ ]:
class Slave(Homo):
    """Sklave mit Bezug auf seinen Herrn (dominus)."""

    def __init__(self, personal_name: str, master: Citizen):
        super().__init__()
        self.personal_name = personal_name
        self.master = master

    @property
    def status(self) -> str:
        return "servus"

    @property
    def full_name(self) -> str:
        # Vereinfachte Form: Tiro servus Marci Tullii Ciceronis wäre korrekt komplexer;
        # wir zeigen hier das Grundprinzip mit dem Namen des Herrn.
        return f"{self.personal_name} servus {self.master.full_name}"

    def free(self, praenomen: str | None = None) -> "Freedman":
        """Manumissio: gibt einen Freedman zurück.

        - praenomen: Praenomen des Freigelassenen (oft das des Patrons)
        """
        new_praenomen = praenomen or self.master.praenomen
        return Freedman(
            praenomen=new_praenomen,
            nomen=self.master.nomen,
            slave_name=self.personal_name,
            patron=self.master,
        )

In [ ]:
class Freedman(Citizen):
    """Freigelassener (libertus) mit Bezug auf den Patron.

    Namensschema (vereinfacht):
    Praenomen + Nomen gentile + Praenomen des Patrons im Genitiv + liberti + Sklavenname
    Beispiel: Marcus Tullius Marci liberti Tiro
    """

    def __init__(self, praenomen: str, nomen: str, slave_name: str, patron: Citizen):
        super().__init__(praenomen=praenomen, nomen=nomen, cognomen=slave_name)
        self.patron = patron
        self.slave_name = slave_name

    @property
    def status(self) -> str:
        return "libertus"

    @property
    def full_name(self) -> str:
        gen = praenomen_genitive(self.patron.praenomen)
        # Marcus Tullius Marci liberti Tiro
        return f"{self.praenomen} {self.nomen} {gen} liberti {self.slave_name}"

## 2. Beispiel: Marcus Tullius Cicero und Tiro

Wir bilden jetzt das im Auftrag genannte Beispiel nach:

In [ ]:
# Zuerst den Herrn definieren: Marcus Tullius Cicero als Bürger
cicero = Citizen(praenomen="Marcus", nomen="Tullius", cognomen="Cicero")
cicero

In [ ]:
# Tiro als Sklave Ciceros
tiro_servus = Slave(personal_name="Tiro", master=cicero)
tiro_servus.full_name, tiro_servus.status

In [ ]:
# Freilassung: Tiro wird libertus des Marcus Tullius Cicero
tiro_libertus = tiro_servus.free()
tiro_libertus.full_name, tiro_libertus.status

Die Ausgabe sollte (sinngemäß) zeigen:

- Sklave: `Tiro servus Marcus Tullius Cicero`
- Freigelassener: `Marcus Tullius Marci liberti Tiro`

Damit ist der Namenswechsel beim Übergang vom Sklaven zum Freigelassenen korrekt modelliert.

## 3. Kinder von Freigelassenen

Kinder eines Freigelassenen sind rechtlich **freie Bürger** (*ingenui*),
nicht mehr *liberti*. Das können wir mit der bestehenden
Citizen-Klasse ausdrücken:

In [ ]:
# Kind eines Freigelassenen: voll freier Bürger
son_of_tiro = Citizen(praenomen="Publius", nomen="Tullius", cognomen="Tiro")
son_of_tiro.full_name, son_of_tiro.status

Hier sieht man:
- Der Sohn trägt das Nomen der Gens seines Vaters
- Sein Status ist `civis Romanus` (freier Bürger)

Damit bildet die Klassenhierarchie die soziale Struktur sauberer ab
als ein vereinfachtes Schema mit einer flachen Kategorie `Libertini`.